# Closure

In [1]:
%load_ext memory_profiler
%load_ext snakeviz
%load_ext cython
%load_ext autoreload
%autoreload 2


from IPython.core import debugger
ist = debugger.set_trace

In [2]:
from py.typyMagics import *
ipy = get_ipython()
ipy.register_magics(typyMagics)

In [3]:
import sys
sys.path.insert(0,'../')

## Base Closure

In [4]:
%%run_and_write ../typyPRISM/closure/Closure.py
import numpy as np
class Closure:
    '''Baseclass for all closures 
    
    .. warning::
    
        Currently, this class doesn't do anything besides group all of the
        closures under a single inheritance heirarchy. This will likely
        change as needs arise.
    '''
        

Overwriting ../typyPRISM/closure/Closure.py


In [5]:
%%run_and_write ../typyPRISM/closure/AtomicClosure.py
import numpy as np
class AtomicClosure:
    '''Baseclass for all atomic closures
    
    .. warning::
    
        Currently, this class doesn't do anything besides group all of the
        *atomic* closures under a single inheritance heirarchy. This will 
        likely change as needs arise.
    '''
        

Overwriting ../typyPRISM/closure/AtomicClosure.py


## PercusYevick

In [6]:
%%run_and_write ../typyPRISM/closure/PercusYevick.py
from typyPRISM.closure.AtomicClosure import AtomicClosure
import numpy as np
class PercusYevick(AtomicClosure):
    '''Percus Yevick closure written in terms of a change of variables
    
    .. note::
    
        The change of variables is necessary in order to use potentials with
        hard cores. Written in the standard form, this closure diverges with
        divergent potentials, which makes it impossible to numerically solve. 
    
    .. math::
        
        c_{i,j}(r) = (exp(-U_{i,j}(r)) - 1.0) * (1.0 + \gamma_{i,j}(r))
        
        \gamma_{i,j}(r) =  h_{i,j}(r) - c_{i,j}(r)
    
    '''
    def __init__(self):
        self.potential = None
        self.value = None
        
    def __repr__(self):
        return '<AtomicClosure: PercusYevick>'
    
    def calculate(self,gamma):
        
        assert self.potential is not None,'Potential for this closure is not set!'
        
        assert len(gamma) == len(self.potential),'Domain mismatch!'
        
        self.value = (np.exp(-self.potential)-1.0)*(1.0+gamma)
        
        return self.value
        
        

Overwriting ../typyPRISM/closure/PercusYevick.py


In [7]:
%%run_and_write ../typyPRISM/test/PercusYevick_TestCase.py
from typyPRISM.closure import PercusYevick
import unittest
import numpy as np

class PercusYevick_TestCase(unittest.TestCase):
    def test_create(self):
        '''Can we create a PercusYevick closure ?'''
        r = np.arange(0.75,3.5,0.05)
        U = np.zeros_like(r)
        U[r<1.0] = 1e6
        
        gamma = np.zeros_like(r)
        C1 = (np.exp(-U) - 1.0)*(1.0 + gamma)
        
        
        PY = PercusYevick()
        PY.potential = U
        C2 = PY.calculate(gamma)
        np.testing.assert_array_almost_equal(C1,C2)
        
        


Overwriting ../typyPRISM/test/PercusYevick_TestCase.py


## RUN ALL

In [8]:
import unittest
suite = []
suite.append(unittest.TestLoader().loadTestsFromTestCase(PercusYevick_TestCase))
suite = unittest.TestSuite(suite)
unittest.TextTestRunner(verbosity=2).run(suite)

test_create (__main__.PercusYevick_TestCase)
Can we create a PercusYevick closure ? ... ok

----------------------------------------------------------------------
Ran 1 test in 0.003s

OK


<unittest.runner.TextTestResult run=1 errors=0 failures=0>